In [120]:

import requests
import http.client

import os
from dotenv import load_dotenv

import datetime

from bs4 import BeautifulSoup
import re
from fuzzywuzzy import fuzz

import spotipy
from spotipy.oauth2 import SpotifyOAuth


In [121]:
# get and format today's date
today = datetime.datetime.today().date().strftime('%Y-%m-%d')
today

'2022-01-30'

In [122]:


h = http.client.HTTPConnection("www.bandsintown.com")

h.request("GET", "/")

my_headers = h.getresponse()

print(my_headers.headers)

Date: Mon, 31 Jan 2022 07:42:38 GMT
Content-Type: text/html
Content-Length: 185
Connection: keep-alive
Server: nginx/1.13.1
Location: https://www.bandsintown.com/
Strict-Transport-Security: max-age=31536000
X-Correlation-ID: 8157585c7d88d40fb477c3f67a431258
X-Request-ID: 8157585c7d88d40fb477c3f67a431258




In [123]:
 # enter website info
url = f'https://www.bandsintown.com/?date={today}&date_filter=This+Week'
headers = {'User-Agent': 'USERAGENT'}

#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15'}

response = requests.get(url, headers=headers)

In [124]:
# check to see if we have soup...  ('200' means that we have a valid connection)
response.status_code

200

In [125]:
# start the soup

soup = BeautifulSoup(response.text, 'html.parser')

In [126]:
# make soup outta that website!

soup.findAll('a')

[<a class="_1c0VJLyfwWzjJfOHu6eP_U" href="https://www.bandsintown.com/?came_from=257"><img alt="Bandsintown" src="data:image/svg+xml,%3C?xml version='1.0' encoding='UTF-8' standalone='no'?%3E %3Csvg version='1.1' id='svg9' width='190' height='39' viewBox='0 0 190 39' xmlns:xlink='http://www.w3.org/1999/xlink' xmlns='http://www.w3.org/2000/svg' xmlns:svg='http://www.w3.org/2000/svg'%3E %3Cdefs id='defs13' /%3E %3Cg id='g15'%3E %3Cimage width='190' height='39' preserveAspectRatio='none' style='image-rendering:optimizeQuality' xlink:href='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAABdwAAAEzCAYAAAAraMEgAAAACXBIWXMAABYvAAAWLwESJu8cAAAg AElEQVR4nO3d0XHcRtY2YGDL9/QfgbQBuCTf+FbcCKSNQHQEpiMQHYHpCCxFYCkCU7d7s1JtACtF 8EkR4C9oz3hhLilxZk4D3cDzVLH8bX02iQEGje4XBwf9MAwdAAAAAABwnL/YfwAAAAAAcDyBOwAA AAAAJBC4AwAAAABAAoE7AAAAAAAkELgDAAAAAEACgTsAAAAAACQQuAMAAAAAQAKBOwAAAAAAJBC4 AwAAAABAAoE7AAAAAAAkELgDAAAAAEACgTsAAAAAACQQuAMAAAAAQAKBOwAAAAAAJBC4AwAAAABA AoE7AAAAAAAkELgDAAAAAEACgTsAAAAAACQQuAMAAAAAQAKBOwA

In [127]:
# find and inspect first element from 'event's near' info from bottom of website

event_urls = soup.findAll('a', '_1cFGVRJP0Z1CIBapvtx7fc')

event_urls[0]

<a class="_1cFGVRJP0Z1CIBapvtx7fc" href="https://www.bandsintown.com/e/103133211-michael-delara-at-arnold's-beach-bar?came_from=257&amp;utm_medium=web&amp;utm_source=home&amp;utm_campaign=event"><div class="_4gLnF-ghHSp52qE2F86u9"><div class="LazyLoad" style="height:110px;width:110px"></div></div><div class="_1moSXo_rZJq4YjrhvXo08f"><div class="BU3JSLt5MOv3NaT1kG8z-"><div class="_3gS4-NmTCW4Tx1vzP4MDW2">Michael DeLara</div><div class="_2gvhOWRdqDe26QDJbCwa-4">Arnold's Beach Bar</div></div><div class="_1014a_eTIRVouGrph7O2Jn"><img alt="calendarIcon" class="_1nln9m64BlBqg8osBbShas" src="https://assets.prod.bandsintown.com/images/newHomeIcon/whiteCalendarIcon.svg"/><div>JAN 31 - 6:00 PM</div></div><div class="_1cuFrQ6SHTFz6KdZCQUnLo"></div></div></a>

In [128]:
# grab the 'event's near' info from bottom of website

pattern = re.compile(r"\b\S*-at\b")

reg_list = re.findall(pattern , str(event_urls))

print(reg_list)

['href="https://www.bandsintown.com/e/103133211-michael-delara-at', 'href="https://www.bandsintown.com/e/103088333-mike-izon-music-at', 'href="https://www.bandsintown.com/e/103131655-fia-at', 'href="https://www.bandsintown.com/e/1023745636-chinese-new-year-at', 'href="https://www.bandsintown.com/e/103088336-mike-izon-music-at', 'href="https://www.bandsintown.com/e/103109597-christafari-at', 'href="https://www.bandsintown.com/e/103131748-jasmin-nicole-at', 'href="https://www.bandsintown.com/e/103131842-jasmin-nicole-at', 'href="https://www.bandsintown.com/e/103124343-bobby-moderow-jr.-at', 'href="https://www.bandsintown.com/e/103048477-christafari-at', 'href="https://www.bandsintown.com/e/1023594950-ron-white-at', 'href="https://www.bandsintown.com/e/1023779564-thunderstorm-artis-at', 'href="https://www.bandsintown.com/e/1023446740-nero-at', 'href="https://www.bandsintown.com/e/103131862-maile-at', 'href="https://www.bandsintown.com/e/103048494-christafari-at', 'href="https://www.bandsi

In [129]:
# select the link from the original grab

for i, b in enumerate(reg_list) :

    reg_list[i] = b.replace('-at','').replace('-', ' ')

reg_list

['href="https://www.bandsintown.com/e/103133211 michael delara',
 'href="https://www.bandsintown.com/e/103088333 mike izon music',
 'href="https://www.bandsintown.com/e/103131655 fia',
 'href="https://www.bandsintown.com/e/1023745636 chinese new year',
 'href="https://www.bandsintown.com/e/103088336 mike izon music',
 'href="https://www.bandsintown.com/e/103109597 christafari',
 'href="https://www.bandsintown.com/e/103131748 jasmin nicole',
 'href="https://www.bandsintown.com/e/103131842 jasmin nicole',
 'href="https://www.bandsintown.com/e/103124343 bobby moderow jr.',
 'href="https://www.bandsintown.com/e/103048477 christafari',
 'href="https://www.bandsintown.com/e/1023594950 ron white',
 'href="https://www.bandsintown.com/e/1023779564 thunderstorm artis',
 'href="https://www.bandsintown.com/e/1023446740 nero',
 'href="https://www.bandsintown.com/e/103131862 maile',
 'href="https://www.bandsintown.com/e/103048494 christafari',
 'href="https://www.bandsintown.com/e/103131685 chante',

In [130]:
# select the band name after the space
# this should work even for bands that start with numbers!

pattern = re.compile(r"\s(.*)")

for i, b in enumerate(reg_list):

    reg_list[i] = re.findall(pattern, str(b))

reg_list

[['michael delara'],
 ['mike izon music'],
 ['fia'],
 ['chinese new year'],
 ['mike izon music'],
 ['christafari'],
 ['jasmin nicole'],
 ['jasmin nicole'],
 ['bobby moderow jr.'],
 ['christafari'],
 ['ron white'],
 ['thunderstorm artis'],
 ['nero'],
 ['maile'],
 ['christafari'],
 ['chante'],
 ['thunderstorm artis'],
 ['christafari']]

In [131]:
# get rid of parentheses and commas (and nested lists by default)

for i, b in enumerate(reg_list) :

    reg_list[i] = b[0].replace("'",'').replace(',', '')

reg_list

['michael delara',
 'mike izon music',
 'fia',
 'chinese new year',
 'mike izon music',
 'christafari',
 'jasmin nicole',
 'jasmin nicole',
 'bobby moderow jr.',
 'christafari',
 'ron white',
 'thunderstorm artis',
 'nero',
 'maile',
 'christafari',
 'chante',
 'thunderstorm artis',
 'christafari']

In [132]:
# drop duplicates by using a set

band_list = list(set(reg_list))

band_list

['fia',
 'chante',
 'christafari',
 'jasmin nicole',
 'maile',
 'ron white',
 'thunderstorm artis',
 'mike izon music',
 'nero',
 'chinese new year',
 'michael delara',
 'bobby moderow jr.']

In [133]:
load_dotenv()
# grab environment variables
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

In [134]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope="playlist-modify-public",
        redirect_uri="https://www.spotify.com/",
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        show_dialog=True,
        cache_path="token.txt",

    )
)
user_id = sp.current_user()["id"]

In [135]:
# Creating a new Playlist
playlist = sp.user_playlist_create(
    user=user_id,
    name=f"Upcoming Local Shows Playlist",
    public=True,
    collaborative=False,
    description=f"Songs from artists that will be playing in your area soon",
)

playlist_id = playlist["id"]

In [143]:
def grab_track_ids(band_list):

    track_ids = []

    for i in range(len(band_list)):
        results = sp.search(q=f"{band_list[i]}", limit=5, type='track')
        if results['tracks']['total'] == 0:
            continue
        else:
            for j in range(len(results['tracks']['items'])):
                if fuzz.partial_ratio(results['tracks']['items'][j]['artists'][0]['name'], band_list[i]) > 75:
                    track_ids.append(results['tracks']['items'][j]['id'])
                else:
                    continue
    print("Gottem!")

    return track_ids

In [144]:
track_ids = grab_track_ids(band_list)

track_ids

Gottem!


['2ZlaPNCTkfbyOn29z3DrSo',
 '5yM15NgJhQw2A3S1LWf5kU',
 '6aBhKc5wZiR8RMgNsc00l0',
 '35WSolh1yQFZYiCqSbqSSl',
 '4pYhtE56PsIbPYLCBTE6T4',
 '07M5jP5WGeCLWhpRVZ7ouL',
 '1DANjn6DqWd98WIR6JfkSp',
 '0eJ55ewXpVQuRqFG8keqf2',
 '1rwRKz3R4HSr6uiARC6gTT',
 '4gXNXQNw922og1kBag6beN',
 '7cXBp4HwwW0IFb6bq6X0p0',
 '63NCB9nUsL8acHNk51klHy',
 '24UCnQe9X3v81Ys3StQK1l',
 '19KdFbzFLAuZN8MCOP8tCi',
 '1gYfwTmrdMapmkQg58JoW7',
 '0ssLwE3jh5gEeFQ7gfhA7j',
 '0B2H6GWmg8KpBv8qh7uO1S',
 '0a40dbixRhznG359jQSivU']

In [141]:
sp.user_playlist_add_tracks(user_id, playlist_id, track_ids)

{'snapshot_id': 'NiwzYzFjZjY4ZThlOTg1ODc1Nzk3NGI5NGJjNDA1NzlkYWJmYTQ5OTQz'}

In [145]:
list_of_playlists = sp.user_playlists(user_id, limit=50, offset=0)

for p in list_of_playlists['items']:

    if p['name'] == 'Upcoming Local Shows Playlist':
        sp.user_playlist_replace_tracks(user_id, playlist_id, track_ids)
        break
    else:
        sp.user_playlist_add_tracks(user_id, playlist_id, track_ids)
        break
